In [26]:
import pandas as pd

In [27]:
df = pd.read_csv("../data-analysis-pipeline/Input/results.csv")
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [28]:
df.dtypes

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object

In [29]:
df['date'] = pd.to_datetime(df['date'])

In [30]:
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [31]:
df = df.loc[df["year"] >= 1990]

In [32]:
coleliminar = ['date','city','country', 'neutral', 'day', 'month', 'year']
df = df.drop(coleliminar, axis=1)

In [33]:
df.head()

,home_team,away_team,home_score,away_score,tournament
15872,Algeria,Mali,5,0,Friendly
15873,Algeria,Cameroon,3,1,Friendly
15874,Greece,Belgium,2,0,Friendly
15875,Mexico,Argentina,2,0,Friendly
15876,Malawi,Tanzania,2,2,Friendly


In [34]:
def tournament(df):
    listtourn = ['CONCACAF Nations League', 'Gold Cup qualification', 'Oceania Nations Cup', 'Friendly', 'African Cup of Nations qualification', 'UEFA Euro', 'African Nations Championship qualifying', 'UEFA Nations League', 'African Cup of Nations', 'Copa América qualification', 'Gold Cup', 'FIFA World Cup', 'Oceania Nations Cup qualification', 'CONCACAF Nations League qualifying', 'Confederations Cup', 'UEFA Euro qualification', 'Copa América', 'FIFA World Cup qualification']
    return df.loc[df['tournament'].isin(listtourn)]

df = tournament(df)  

In [35]:
def points(row1,row2):
    row = 0
    if row1 > row2:
        row +=3
    elif row1 == row2:
        row +=1
    elif row1 < row2:
        row = 0
    return row

In [36]:
for index, row in df.iterrows():
    df.loc[index,"home_points"] = points(row['home_score'], row['away_score'])
    
for index, row in df.iterrows():
    df.loc[index, "away_points"] = points(row['away_score'], row['home_score'])

In [37]:
df = df[['home_team', 'away_team', 'home_points', 'away_points']]

In [38]:
df[['home_team', 'away_team']] = df[['home_team', 'away_team']].apply(lambda x: x.str.upper())

In [39]:
home_df = df.groupby('home_team', as_index=False).agg({'home_points':'mean'})
home_df = home_df.rename(columns={'home_team':'Country'})
away_df = df.groupby('away_team', as_index=False).agg({'away_points':'mean'})
away_df = away_df.rename(columns={'away_team':'Country'})

In [40]:
definitive = pd.merge(home_df, away_df, on='Country')

In [41]:
definitive['avg_points'] = definitive['home_points'] + definitive['away_points']

In [43]:
definitive = definitive[['Country', 'avg_points']]

In [44]:
definitive.head()

,Country,avg_points
0,ABKHAZIA,1.000000
1,AFGHANISTAN,1.937729
2,ALBANIA,2.127632
3,ALGERIA,3.037380
4,AMERICAN SAMOA,0.823864


In [49]:
lista = ['ANDALUSIA', 'BASQUE COUNTRY', 'KERNOW', 'ARTSAKH', 'YORKSHIRE', 'YUGOSLAVIA', 'ABKHAZIA', 'JERSEY', 'CZECHOSLOVAKIA', 'MICRONESIA', 'GERMAN DR']

In [50]:
definitive = definitive.loc[(~definitive['Country'].isin(lista))]

In [51]:
definitive = definitive.sort_values(by='avg_points', ascending=False)

In [58]:
definitive = definitive.reset_index()
definitive = definitive[['Country', 'avg_points']]
definitive['Position'] = definitive.index + 1
definitive = definitive[['Position', 'Country', 'avg_points']]

In [59]:
definitive

,Position,Country,avg_points
0,1,SPAIN,4.399058
1,2,BRAZIL,4.295744
2,3,FRANCE,4.132602
3,4,GERMANY,4.083409
4,5,PORTUGAL,3.890548
...,...,...,...
227,228,ANGUILLA,0.443850
228,229,EAST TIMOR,0.428571
229,230,ANDORRA,0.422602
230,231,SAN MARINO,0.086728
